# Feedforward Neural Network from scratch for Text Classification

### Contents

1. Goal
2. Approach
3. Dataset

### 1. Goal

- The goal of this notebook is to develop a Feedforward Neural Netowork from scratch for text classification.



### 2. Approach

- Transform raw text data into input vectors


- Implement Feedforward network with
    - Input: Embedding weight matrix
    - Interim: Hidden layer(S)
    - Output: softmax
    
    
- Implement Stochastic Gradient Descent (SGD) algorithm with
    - Forward pass to compute intermediate outputs
    - Backward pass (Backpropagation) to compute gradients and update weights of the network
    - Dropout for regularisation
    - Perform hyperparater tuning
        - dropout rate
        - embedding size
        - Learning rate
    - Plot the learning process
        - training loss
        - validation loss
        
        
- Implement pre-trained word embeddings (GloVe)
    - Reinitialise network weights with pre-trained word embeddings from GloVe
    - Don't update weight during training (weight freezing, NO backpropagation)
    - Perform hyperparameter tuning
    - Plot the learning process
    
    
- Implement More hidden layers


### 3. Dataset

#### AG News Corpus
- The data is taken from [AG News Corpus](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html).
    - Created folder structure:
        - `data/train.csv`: contains 2,400 news articles, 800 for each class to be used for training.
        - `data/dev.csv`: contains 150 news articles, 50 for each class to be used for hyperparameter selection and monitoring the training process.
        - `data/test.csv`: contains 900 news articles, 300 for each class to be used for testing.

#### GloVe Embeddings

- The pre-trained GloVe embeddings trained on Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download) is available from [here](http://nlp.stanford.edu/data/glove.840B.300d.zip).


In [1]:
# Important important python libraries

import pandas as pd
import numpy as np
import random
import re
from collections import Counter
from tqdm import tqdm
from tqdm.notebook import tnrange

In [2]:
# Set random seed for reproducibility
def set_seed():
    random.seed(123)
    np.random.seed(123)
    
set_seed()

### Transform raw text data into input vectors

- Transform the raw text into train, dev, and test sets
- Since the dataset done't have headers, we will first create custome headers and then load the data

In [3]:
# Create a customer Header
news_header = ["label", "text"]

# Load news data
news_train = pd.read_csv('./data/train.csv', names=news_header)
news_dev = pd.read_csv('./data/dev.csv', names=news_header)
news_test = pd.read_csv('./data/test.csv', names=news_header)


In [4]:
# Check the data
news_train.head(5)

,label,text
0,1,Reuters - Venezuelans turned out early\and in ...
1,1,Reuters - South Korean police used water canno...
2,1,Reuters - Thousands of Palestinian\prisoners i...
3,1,AFP - Sporadic gunfire and shelling took place...
4,1,AP - Dozens of Rwandan soldiers flew into Suda...


In [5]:
news_dev.head(5)

,label,text
0,1,"BAGHDAD, Iraq - An Islamic militant group that..."
1,1,Parts of Los Angeles international airport are...
2,1,AFP - Facing a issue that once tripped up his ...
3,1,The leader of militant Lebanese group Hezbolla...
4,1,JAKARTA : ASEAN finance ministers ended a meet...


In [6]:
news_test.head(5)

,label,text
0,1,Canadian Press - VANCOUVER (CP) - The sister o...
1,1,AP - The man who claims Gov. James E. McGreeve...
2,1,"NAJAF, Iraq - Explosions and gunfire rattled t..."
3,1,"LOURDES, France - A frail Pope John Paul II, b..."
4,1,Supporters and rivals warn of possible fraud; ...


Separate the features and lables lists for train, dev, and test sets

In [7]:
news_Xtrain = news_train['text'].tolist()
news_Xdev = news_dev['text'].tolist()
news_Xtest = news_test['text'].tolist()

news_ytrain = news_train['label'].to_numpy()
news_ydev = news_dev['label'].to_numpy()
news_ytest = news_test['label'].to_numpy()


In [8]:
# Check the size of the new datasets
print(len(news_Xtrain))
print(len(news_ytrain))
print(len(news_Xdev))
print(len(news_ydev))
print(len(news_Xtest))
print(len(news_ytest))

2400
2400
150
150
900
900


### Implement Feedforward network

**Create input representations**
- To build a Feedforward neural network, the input data representation needs to be obtained in two ways; from vocabulary or from one-hot encoding. Since, one-hot encoding reqires large memory capacity, we ill repreent the input documents as a list of vocabulary indices where each wrod corresponds to a vocabulary index.

**Text pre-processing pipeline**
- To obtrain the vocabulary of words, we will create text pre-processing pipeline as below:
    - Tokenise all texts into a list of unigrams
    - Remove stop words
    - Remove unigrams appearing in less than K documents
    - Use the remaining unigrams to create the vocabulary of top-N most frequent unigrams in the entire corpus

In [9]:
# Create the stop words list
stop_words = ['a', 'i', "i'm", 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 
              'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
              'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
              'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
              'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
              'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
              'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 
              'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 
              'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 
              'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
              'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 
              'each', 'few', 'more', 'most', 'other', 'some', 'such', 
              'only', 'own', 'same', 'so', 'than', 'too', 'very', 'can', 'will', 
              'just', 'should', "should've", 'now', 'll', 're', 
              've', 'ma', ".", ",", "would", "could", "must", "shall", "may"]

### Unigram extraction from a document


Implement ngram extraction function `extract_ngrams` where 
- `x_raw`: a string corresponding to the raw text of a document
- `ngram_range`: a tuple of two integers denoting the type of ngrams being extracted (e.g. (1,2) denotes extracting unigrams and bigrams)
- `token_pattern`: a string to be used within a regular expression to extract all tokens. Note that the data is already tokenised so we will use white space tokenisation
- `stop_words`: a list of stop words
- `vocab`: a given vocabulary. It should be used to extract specific features
    
and return 
- list of all extracted features    

In [10]:
# Implement ngram extraction
def extract_ngrams(x_raw, ngram_range=(1,3), 
                  token_pattern=f'\b[A-Za-z][A-Za-z]+\b',
                 stop_words=[], vocab=set()):
    token_regex = re.compile(token_pattern)
    
    # Extract all unigrams by tokenising
    x_unigram = [w for w in token_regex.findall(str(x_raw).lower(),) if w not in stop_words]
    
    # Store the unigrams to be returned
    x = []
    if ngram_range[0] == 1:
        x = x_unigram
    
    # Generate n-grams from avaialbe unigrams
    ngrams = []
    for n in range(ngram_range[0], ngram_range[1]+1):
        if n==1: 
            continue
        
        # Pass a list of lists as an argument for zip
        arg_list = [x_unigram]+[x_unigram[i:] for i in range(1, n)]
        
        # extract tuples of n-grams using zip
        # for bigram this should look: list(zip(x_uni, x_uni[1:]))
        # align each item x[i] in x_uni with the next one x[i+1]. 
        # Note that x_uni and x_uni[1:] have different lenghts
        # but zip ignores redundant elements at the end of the second list
        # Alternatively, this could be done with for loops
        x_ngram = list(zip(*arg_list))
        ngrams.append(x_ngram)
    
    for n in ngrams:
        for t in n:
            x.append(t)
            
    if len(vocab)>0:
        x=[w for w in x if w in vocab]
    
    return x



### Create a vocabular of n-grams

Implement `get_vocab` function, where
- `X_raw`: a list of strings each corresponding to the raw text of a document
- `ngram_range`: a tuple of two integers denoting the type of ngrams being extracted (e.g. (1,2) denotes extracting unigrams and bigrams)
- `token_pattern`: a string to be used within a regular expression to extract all tokens. Note that data is already tokenised
- `stop_words`: a list of stop words
- `min_df`: keep ngrams with a minimum document frequency
- `keep_topN`: keep top-N more frequent ngrams

and return
- `vocab`: a set of n-grams that will be used as features
- `df`: a document frequency dictionay that contains ngrams as key and their corresponding document frequenciy as values
- `ngram_counts`: counts of each ngram in vocab

In [11]:
def get_vocab(X_raw, ngram_range=(1,3),
             token_pattern=r'\b[A-Za-z][A-Za-z]+\b',
             min_df=0, keep_topN=0, stop_words=[]):

    token_regex = re.compile(token_pattern)
    df = Counter()
    ngram_counts = Counter()
    vocab = set()
    
    # Iterate through each raw text
    for x in X_raw:
        x_ngram = extract_ngrams(x, ngram_range=ngram_range, 
                                 token_pattern=token_pattern, 
                                 stop_words=stop_words)
        # Update document frequency and ngram count
        df.update(list(set(x_ngram)))
        ngram_counts.update(x_ngram)
        
    # Obtain vocabulary as a set with document frequency > min document frequency
    vocab = set([w for w in df if df[w]>=min_df])
    
    # Keep the top N most frequent vocab
    if keep_topN:
        vocab =set([w[0] for w in ngram_counts.most_common(keep_topN)
                   if w[0] in vocab])
        
    return vocab, df, ngram_counts
        

### Get the vocab

In [12]:
keep_topN = 2000
ngram_range = (1,1)
min_df = 0
(vocab_set,
df,
ngram_counts) = get_vocab(news_Xtrain, ngram_range=ngram_range,
                          min_df=min_df, keep_topN=keep_topN,
                          stop_words=stop_words)
news_vocab = list(vocab_set)
print(f'news vocab size: {len(news_vocab)}')
print(news_vocab)


news vocab size: 2000
['congolese', 'face', 'mobile', 'ibm', 'warning', 'tools', 'gained', 'anticipated', 'refugee', 'kostas', 'port', 'fall', 'friends', 'looking', 'remote', 'right', 'activists', 'wild', 'villa', 'mart', 'jamaica', 'weekly', 'improvements', 'prosecutors', 'bowl', 'resolve', 'row', 'presidential', 'fear', 'quarter', 'believe', 'mistakes', 'general', 'profit', 'got', 'williams', 'problems', 'rain', 'whether', 'pull', 'northern', 'debut', 'michael', 'vijay', 'olympia', 'secretary', 'assault', 'google', 'inventory', 'beating', 'abroad', 'teams', 'america', 'scheduled', 'looks', 'rather', 'sad', 'heard', 'output', 'former', 'program', 'insurer', 'lourdes', 'operator', 'hospitals', 'company', 'historic', 'calling', 'black', 'singh', 'baseman', 'katerina', 'inflationary', 'fuel', 'crowd', 'prince', 'candidate', 'bell', 'cleric', 'affair', 'safety', 'illinois', 'cowboys', 'let', 'indian', 'scotland', 'separate', 'kid', 'injured', 'lines', 'services', 'need', 'pitched', 'east'

As far as vocabulary size is concerned, the larger vocabulary gives larger breadth of words to be picked up during training. However, for this experiment, I have restricted to size 2000. In the upcoming section, we will do hyperparameter tuning during training to get the reasonable accuracy while using this vocabulary.

In [13]:
# Create word to id dictionary for future use
word2id = dict(zip(news_vocab, range(len(news_vocab))))

### Convert the list of unigrams  into a list of vocabulary indices
- Storing actual one-hot vectors into memory for all words in the entire data set is prohibitive. Instead, we will store word indices in the vocabulary and look-up the weight matrix. This is equivalent of doing a dot product between an one-hot vector and the weight matrix. 

- Implement a function `extract_ngrams_for_train_dev_test_sets` to represent documents in train, dev and test sets as lists of words in the vocabulary, where
    - `ngram_range`: a tuple of two integers denoting the type of ngrams being extracted (e.g. (1,2) denotes extracting unigrams and bigrams)
    - `vocab`: a vocabulary set
- and returns
    - `Xtrain_ngrams`: ngrams for train set
    - `Xdev_ngrams`: ngrams for dev set
    - `Xtest_ngrams`: ngrams for test set
    

In [14]:
def extract_ngrams_for_train_dev_test_sets(ngram_range, vocab):
    Xtrain = news_Xtrain
    Xdev   = news_Xdev
    Xtest  = news_Xtest
    
    # Extract n-grams for training set
    Xtrain_ngrams = list()
    for i in tnrange(len(Xtrain)):
        Xtrain_ngrams.append(extract_ngrams(Xtrain[i], 
                                            ngram_range=ngram_range, token_pattern=r'\b[A-Za-z][A-Za-z]+\b',
                                            stop_words=stop_words, vocab=vocab))
    # Extract n-grams for development set
    Xdev_ngrams = list()
    for i in tnrange(len(Xdev)):
        Xdev_ngrams.append(extract_ngrams(Xdev[i], 
                                            ngram_range=ngram_range, token_pattern=r'\b[A-Za-z][A-Za-z]+\b',
                                            stop_words=stop_words, vocab=vocab))
    # Extract n-grams for test set
    Xtest_ngrams = list()
    for i in tnrange(len(Xtest)):
        Xtest_ngrams.append(extract_ngrams(Xtest[i], 
                                            ngram_range=ngram_range, token_pattern=r'\b[A-Za-z][A-Za-z]+\b',
                                            stop_words=stop_words, vocab=vocab))
        
    return Xtrain_ngrams, Xdev_ngrams, Xtest_ngrams

In [15]:
(news_Xtrain_ngrams, 
 news_Xdev_ngrams, 
 news_Xtest_ngrams) = extract_ngrams_for_train_dev_test_sets(ngram_range, news_vocab)


  0%|          | 0/2400 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/900 [00:00<?, ?it/s]

In [16]:
# Convert into list of indices
news_Xtrain_ngrams_indices = []
for ngrams in news_Xtrain_ngrams:
    news_Xtrain_ngrams_indices.append([word2id.get(ngram, -1) for ngram in ngrams])
    
news_Xdev_ngrams_indices = []
for ngrams in news_Xdev_ngrams:
    news_Xdev_ngrams_indices.append([word2id.get(ngram, -1) for ngram in ngrams])

news_Xtest_ngrams_indices = []
for ngrams in news_Xtest_ngrams:
    news_Xtest_ngrams_indices.append([word2id.get(ngram, -1) for ngram in ngrams])

In [17]:
# check the sample of ngram and corresponging vocabulary index
print(news_Xtrain_ngrams[0])
print(news_Xtrain_ngrams_indices[0])

['reuters', 'venezuelans', 'turned', 'early', 'large', 'numbers', 'sunday', 'vote', 'historic', 'referendum', 'either', 'left', 'wing', 'president', 'hugo', 'chavez', 'office', 'give', 'new', 'mandate', 'next', 'two', 'years']
[1889, 1483, 163, 1379, 1577, 1655, 1714, 491, 66, 441, 1573, 1707, 295, 1878, 1650, 575, 1727, 1919, 1534, 1096, 1243, 965, 1460]


# Feedforward Neural Network Architecture

The steps to create Feedforward neural network as as following:

- Pass each word index into its corresponding embedding by looking-up on the embedding matrix and then compute the first hidden layer $\mathbf{h}_1$:

$$\mathbf{h}_1 = \frac{1}{|x|}\sum_i W^e_i, i \in x$$

where $|x|$ is the number of words in the document and $W^e$ is an embedding matrix $|V|\times d$, $|V|$ is the size of the vocabulary and $d$ the embedding size.

- Then $\mathbf{h}_1$ should be passed through a ReLU activation function:

$$\mathbf{a}_1 = relu(\mathbf{h}_1)$$

- Finally the hidden layer is passed to the output layer:


$$\mathbf{y} = \text{softmax}(\mathbf{a}_1W) $$ 
where $W$ is a matrix $d \times |{\cal Y}|$, $|{\cal Y}|$ is the number of classes.

- During training, $\mathbf{a}_1$ is going to be multiplied with a dropout mask vector (elementwise) for regularisation before it is passed to the output layer.

- The network will also be extended to a deeper architecture by passing a hidden layer to another one:

$$\mathbf{h_i} = \mathbf{a}_{i-1}W_i $$

$$\mathbf{a_i} = relu(\mathbf{h_i}) $$



Create a new function called `get_weight_matrix` which will be used to initialise weight matrix for Feedforward nueral network. The function takes as input:

- `row_size`: The row size of the weight matrix
- `col_size`: The column size of the weight matrix
- `init_val`: The initialisation values that are used to create a range of values from negative to positive

and returns:
- `init_W`: The weight matrix initialised with default values

In [20]:
def get_weight_matrix(row_size, col_size, init_val):
    init_W = np.zeros((row_size, col_size))
    for j in range(row_size):
        weights = np.array(np.random.uniform(-1 * init_val, init_val, col_size)).astype(np.float32)
        init_W[j] = weights
    return init_W


### Network Training

First we need to define the parameters of our network by initialising the weight matrix. To do this, we will create a function `network_weights` that takes inputs:

- `vocab_size`: The size of the vocabulary
- `embedding_dim`: The size of the word embeddings
- `hidden_dim`: The list of the sizes of hidden layers. Empty if there are not hidden layers between the average embedding and the output layer
- `num_classes`: The number of classes for the output layer

and returns:
- `W`: A dictionary mapping from layer index (e.g. 0 for the embedding matrix) to the corresponding weight matrix initialised with small random numbers

Note: We have to make sure that the dimensionality of each weight matrix is compatible with the previous and next weight matrix, otherwise the forward and backward pass won't be able to perform. We will also consider using np.float32 precision to save memory.

In [24]:
def network_weights(vocab_size=1000, embedding_dim=300,
                   hidden_dim=[], num_classes=3, init_val=0.5):
    
    # initialise weights
    W = []
    
    # input layer weight matrix for word embeddings
    W.append({
        "embedding_matrix": get_weight_matrix(vocab_size, embedding_dim, init_val)
    })
    
    # hidden layer weight matrix
    row_size = embedding_dim
    for j in range(len(hidden_dim)):
        col_size = hidden_dim[j]
        W.append({"weights": get_weight_matrix(row_size, col_size, init_val)})
        row_size = col_size
        
    # output layer weight matrix
    col_size = num_classes
    W.append({"weights": get_weight_matrix(row_size, col_size, init_val)})
    
    return W
    
    
    

In [31]:
# sanity check to see the network weight generation and their dimensions
print(network_weights(vocab_size=3, embedding_dim=4, hidden_dim=[2], num_classes=2))

[{'embedding_matrix': array([[ 0.19631127, -0.05967212, -0.06178562,  0.2650961 ],
       [ 0.065642  , -0.41509584,  0.08267109,  0.31484371],
       [-0.16293362,  0.42757657,  0.25071701,  0.07406382]])}, {'weights': array([[ 0.25164399, -0.42085105],
       [ 0.35938907,  0.32150412],
       [ 0.40987167, -0.3713688 ],
       [-0.41821992, -0.36158442]])}, {'weights': array([[-0.10062129, -0.07569314],
       [ 0.06221838, -0.37775645]])}]


### softmax function

Due to floating point limitations in numPy, softmax frequently gets `NaN` and `inf` error. To eliminate this error, a softmax normalisation technique is used by subtracting the maximum value of z from the given z value. This way, softmax is normalised across all values for every forward pass and will not generate the errors.

In [32]:
def softmax(z):
    z = z - np.max(z)
    return np.exp(z) / np.sum(np.exp(z))

### Categorical Loss function

The categorical loss function is used to calculate the objective loss during training.

In [33]:
def categorical_loss(X, Y, W):
    lossse = list()
    num_classes = 3
    for i, x in enumerate(X):
        out_vals = forward_pass(x, W, dropout_rate=0.0)
        
        # one-hot vector to represent the correct class during categorical loss calculation
        y = one_hot_vector(Y[i]-1, num_classes)
        loss = -y * np.log(out_vals['y_pred'].T)
        losses.append(np.sum(loss.T))
    return np.mean(losses)


### Categorical Loss Derivative function

We need to also create the derivative function to calculate the derivative of categorical loss during backward pass (aka. backpropagation).

In [34]:
def categorical_loss_derivative(y, y_pred):
    loss_der = y_preds.T - y
    return loss_der.T

### ReLU and ReLU Derivative function

We will not implement `relu` function to introduce non-liniearity afte hidden layers. This function will at as activation function during forward pass.

$$relu(z_i)= max(z_i,0)$$

And the `relu_derivative` function is used to compute its derivative during backward pass (aka. backpropagation)

  $$relu\_derivative(z_i)=0$$ if $z_i$<=0, 1 otherwise.
  
Note: Both function take as input a vector $z$.

In [35]:
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    dz = np.copy(z)
    dz[np.where(dz>0)] = 1
    dz[np.where(dz<=0)] = 0
    return dzå

A derivative of ReLU is differentiable at all points except at 0. The left derivative at z<0 is 0 and right derivative at z>0 is 1. But derivative at z=0 is technically undefined. In Colloquial terms, the function is not smooth at z=0 and there are many possible lines (slops) we could fit through. So that do we do here? Basically, we can arbitrarily choose a value as 0, 0.5, or 1 but for simplicity we can choose 0 for z=0.

### Dropout mask

During training, we will apply dropout mask element-wise to do model finetuning to avoid overfitting. This masking will be done after the ReLU activation function (i.e. vector of ones with random percentage set to zero).

In [40]:
def dropout_mask(size, dropout_rate):
    dropout_vec = np.ones((size))
    idx = np.random.choice(len(dropout_vec), 
                          size=int(dropout_rate * size), 
                          replace=False)
    dropout_vec[idx] = 0
    return dropout_vec

In [42]:
# sanity check
print(dropout_mask(10, 0.2))
print(dropout_mask(10, 0.2))

[1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
[1. 1. 0. 1. 0. 1. 1. 1. 1. 1.]


Work in progress...